In [1]:
import pandas as pd
import pickle as pkl
import os
import re


In [2]:
df = pd.read_excel("data/annotations/v4-characterized-HIV and TB protocol data.xlsx")

In [3]:
from front_end.processors.country_extractor import CountryExtractor

country_extractor = CountryExtractor()

ModuleNotFoundError: No module named 'util'

In [ ]:
country_extractor.process(["i am in Zimbabwe "])

In [ ]:
input_folder = "data/preprocessed_tika/"
file_to_text = {}
for root, folder, files in os.walk(input_folder):
    for file_name in files:
        if not file_name.endswith("pkl"):
            continue
        pdf_file = re.sub(".pkl", "", file_name)

        full_file = input_folder + "/" + file_name
        #         print (full_file)
        with open(full_file, 'rb') as f:
            text = pkl.load(f)
        file_to_text[pdf_file] = text

In [ ]:
file_to_prediction = {}

In [ ]:
for file, pages in file_to_text.items():
    print (file)
    country_to_pages = {}
    
    for page_no, page_text in enumerate(pages):
        countries = find_countries(page_text)

        demonyms = find_demonyms(page_text)
#         if len(demonyms) > 0:
#             print("DEMO", demonyms)
        countries.extend(demonyms)

        for country, match in countries:
            if country.alpha_2 in allowed_countries:
                if country.alpha_2 not in country_to_pages:
                    country_to_pages[country.alpha_2] = []
                country_to_pages[country.alpha_2].append(page_no)
                
                
    prediction = set()

    if len(country_to_pages) > 0:
        earliest_page = sorted([min(p) for p in country_to_pages.values()])[0]
        for candidate, pages in country_to_pages.items():
            # Any country mentioned on 30+ pages is an investigation country
            unique_pages = set(pages)
            if len(unique_pages) > 30:
                prediction.add(candidate)
            # Any country mentioned on the first contentful page is also a candidate
            if 0 in unique_pages or earliest_page in unique_pages:
                prediction.add(candidate)

        first_mentioned_countries = sorted(country_to_pages.items(), key=lambda a: min(a[1]))
        prediction.add(first_mentioned_countries[0][0])
        
    file_to_prediction[file] = prediction

In [ ]:
df["predicted"] = df["File Name"].map(file_to_prediction)

In [ ]:
# df["predicted"] = df["File Name"].map(file_to_prediction).fillna((df['NCT #'] + ".pdf").map(file_to_prediction))

In [ ]:
df["ground_truth"] = df["Country/ Countries"].apply(find_countries).apply(lambda c : [x[0].alpha_2 for x in c])

In [ ]:
df[["NCT #", "File Name", "Country/ Countries", "ground_truth", "predicted"]]